In [1]:
#importing the dependencies

In [2]:
import tensorflow as tf
import os
import cv2
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

In [3]:
# Define the Residual Block
def residual_block(x, filters, downsample=False):
    identity = x
    stride = 1

    if downsample:
        stride = 2
        identity = layers.Conv2D(filters, 1, strides=2, padding='same')(identity)
        identity = layers.BatchNormalization()(identity)

    x = layers.Conv2D(filters, 3, strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Add()([x, identity])
    x = layers.ReLU()(x)
    return x

# Define the ResNet-18 model
def resnet18(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(64, 7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)

    x = residual_block(x, 64)
    x = residual_block(x, 64)
    x = residual_block(x, 128, downsample=True)
    x = residual_block(x, 128)
    x = residual_block(x, 256, downsample=True)
    x = residual_block(x, 256)
    x = residual_block(x, 512, downsample=True)
    x = residual_block(x, 512)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=x)
    return model

In [4]:
# Specify the path to the dataset folder
dataset_path = 'data'

In [5]:
# Define the classes (subfolder names)
classes = ['Gas Leak', 'No Leak']

In [6]:
# Define the input shape and number of classes
input_shape = (224, 224, 3)
num_classes = len(classes)

In [7]:
# Initialize empty lists to store the data and labels
data = []
labels = []

In [8]:
# Iterate over the classes
for class_label, class_name in enumerate(classes):
    # Get the path to the class folder
    class_path = os.path.join(dataset_path, class_name)
    
    # Iterate over the files in the class folder
    for filename in os.listdir(class_path):
        # Get the path to the file
        file_path = os.path.join(class_path, filename)
        
        # Read the image
        image = cv2.imread(file_path)
        
        # Resize the image to the desired input shape
        image = cv2.resize(image, input_shape[:2])
        
        # Normalize the pixel values between 0 and 1
        image = image.astype('float32') / 255.0
        
        # Append the image to the data list
        data.append(image)
        
        # Append the label to the labels list
        labels.append(class_label)

In [9]:
# Convert the data and labels to numpy arrays
data = np.array(data)
labels = np.array(labels)

In [10]:
# Convert the labels to one-hot encoded vectors
labels = to_categorical(labels)


In [11]:
# Split the data into train, validation, and test sets (e.g., 70%, 15%, 15%)
train_split = 0.7
val_split = 0.15

In [12]:
num_samples = len(data)
num_train_samples = int(train_split * num_samples)
num_val_samples = int(val_split * num_samples)

In [13]:
# Shuffle the data and labels in unison
np.random.seed(42)
shuffle_indices = np.random.permutation(num_samples)
data = data[shuffle_indices]
labels = labels[shuffle_indices]

In [14]:
# Split the data into train, validation, and test sets
x_train = data[:num_train_samples]
y_train = labels[:num_train_samples]
x_val = data[num_train_samples:num_train_samples+num_val_samples]
y_val = labels[num_train_samples:num_train_samples+num_val_samples]
x_test = data[num_train_samples+num_val_samples:]
y_test = labels[num_train_samples+num_val_samples:]



In [15]:
# Create an instance of the ResNet-18 model
model = resnet18(input_shape, num_classes)



In [16]:
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])



In [17]:
# Train the model
model.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_val, y_val))



Epoch 1/50
15/15 [==============================] - 22s 1s/step - loss: 1.8743 - accuracy: 0.6381 - val_loss: 1433.7926 - val_accuracy: 0.6300
Epoch 2/50
15/15 [==============================] - 20s 1s/step - loss: 0.5977 - accuracy: 0.6895 - val_loss: 1451.1875 - val_accuracy: 0.6200
Epoch 3/50
15/15 [==============================] - 20s 1s/step - loss: 0.5726 - accuracy: 0.7259 - val_loss: 372.5378 - val_accuracy: 0.6300
Epoch 4/50
15/15 [==============================] - 20s 1s/step - loss: 0.5699 - accuracy: 0.7002 - val_loss: 100.3555 - val_accuracy: 0.6400
Epoch 5/50
15/15 [==============================] - 20s 1s/step - loss: 0.5434 - accuracy: 0.7302 - val_loss: 32.0913 - val_accuracy: 0.6300
Epoch 6/50
15/15 [==============================] - 19s 1s/step - loss: 0.5457 - accuracy: 0.7495 - val_loss: 10.5309 - val_accuracy: 0.6500
Epoch 7/50
15/15 [==============================] - 19s 1s/step - loss: 0.5142 - accuracy: 0.7666 - val_loss: 2.2220 - val_accuracy: 0.6500
Epoch 8/

In [18]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

4/4 - 1s - loss: 1.4334 - accuracy: 0.3267 - 1s/epoch - 275ms/step
Test Loss: 1.4334405660629272
Test Accuracy: 0.32673266530036926


In [19]:
# Load and preprocess the image
image_path = 'imagesfortest/23.png'
image = cv2.imread(image_path)
image = cv2.resize(image, (224, 224))
image = image.astype('float32') / 255.0
image = np.expand_dims(image, axis=0)

In [20]:
# Predict the category
predictions = model.predict(image)
predicted_class = np.argmax(predictions[0])
confidence = predictions[0][predicted_class]

1/1 [==============================] - 0s 339ms/step


In [21]:
# Get the class label and confidence score
predicted_label = classes[predicted_class]
print("Predicted Label:", predicted_label)
print("Confidence:", confidence)

Predicted Label: No Leak
Confidence: 0.96766794
